In [12]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [13]:
import csv, json, os, math, numbers, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, thread, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [14]:
def mirror_tiles(src_pattern, dest_dir, max_level):
    donefile = dest_dir + '/complete'
    if os.path.exists(donefile):
        print '{donefile} already exists, skipping'.format(**locals())
        return
    already_done = 0
    downloaded = 0
    downloaded_size_gb = 0
    for level in range(0, max_level + 1):
        for x in range(0, 2 ** level):
            for y in range(0, 2 ** level):
                prototile = 'prototiles/{level}/{x}/{y}.bin'.format(**locals())
                if 0 == os.stat(prototile).st_size:
                    # print '{level}/{x}/{y}: prototile empty, skipping'.format(**locals())
                    continue
            
                src = src_pattern.format(**locals())
                dest = dest_dir + '/' + '/'.join(src.split('/')[-3:])
                if os.path.exists(dest):
                    already_done += 1
                    continue
          
        
                try:
                    os.makedirs(os.path.dirname(dest))
                except OSError:
                    pass
            
                data = urllib2.urlopen(src).read()
                nbytes = len(data)
        
                tmpname = dest + '.%d.%d.tmp' % (os.getpid(), thread.get_ident())
                open(tmpname, 'w').write(data)
                os.rename(tmpname, dest)
                downloaded += 1
                downloaded_size_gb += nbytes / 1.0e9
    open(donefile, 'w')
    print '{dest_dir}: newly downloaded {downloaded} tiles, {downloaded_size_gb} GB.  ({already_done} tiles already downloaded) '.format(**locals())


In [15]:
layers = {}

layers['own_rent_2016'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;acs2016_5year_tract2010.B07013_002;;%2300FFFF;acs2016_5year_tract2010.B07013_003'
layers['acs2016_renter_w_children'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ff00;acs2016_5year_tract2010.B25115_018;;%23ff0000;acs2016_5year_tract2010.B25115_022%2Bacs2016_5year_tract2010.B25115_025'

#layers['acs2016_poverty_by_race'] = 'https://dotmaptiles.createlab.org/tilesv1/%23f5f5f5;acs2016_5year_tract2010.B17020H_003%2Bacs2016_5year_tract2010.B17020H_004%2Bacs2016_5year_tract2010.B17020H_005;;%23d94801;acs2016_5year_tract2010.B17020B_003%2Bacs2016_5year_tract2010.B17020B_004%2Bacs2016_5year_tract2010.B17020B_005;;%23f1f823;acs2016_5year_tract2010.B17020I_003%2Bacs2016_5year_tract2010.B17020I_004%2Bacs2016_5year_tract2010.B17020I_005'
layers['acs2016_poverty_by_race'] = 'https://dotmaptiles.createlab.org/tilesv1/%236eaaf1;acs2016_5year_tract2010.B17020H_003%2Bacs2016_5year_tract2010.B17020H_004%2Bacs2016_5year_tract2010.B17020H_005;;%235eff00;acs2016_5year_tract2010.B17020B_003%2Bacs2016_5year_tract2010.B17020B_004%2Bacs2016_5year_tract2010.B17020B_005;;%23ffb300;acs2016_5year_tract2010.B17020I_003%2Bacs2016_5year_tract2010.B17020I_004%2Bacs2016_5year_tract2010.B17020I_005'

    
#layers['total_race_1990'] = 'https://dotmaptiles.createlab.org/tilesv1/%23f1f823;census1990_block2010.P0080001;;%23f5f5f5;census1990_block2010.P0100001;;%23d94801;census1990_block2010.P0100002;;%2300fb21;census1990_block2010.P0100004;;%2367a9cf;census1990_block2010.P0100003;;%23F062FF;census1990_block2010.P0100005'
#layers['total_race_2000'] = 'https://dotmaptiles.createlab.org/tilesv1/%23f1f823;census2000_block2010.P0080010;;%23f5f5f5;census2000_block2010.P0080003;;%23d94801;census2000_block2010.P0080004;;%2300fb21;census2000_block2010.P0080006%2Bcensus2000_block2010.P0080007;;%2367a9cf;census2000_block2010.P0080005;;%23F062FF;census2000_block2010.P0080008%2Bcensus2000_block2010.P0080009'
#layers['total_race_2010'] = 'https://dotmaptiles.createlab.org/tilesv1/%23f1f823;census2010_block2010.P0090002;;%23f5f5f5;census2010_block2010.P0090005;;%23d94801;census2010_block2010.P0090006;;%2300fb21;census2010_block2010.P0090008%2Bcensus2010_block2010.P0090009;;%2367a9cf;census2010_block2010.P0090007;;%23F062FF;census2010_block2010.P0090010%2Bcensus2010_block2010.P0090009'
#layers['total_race_2016'] = 'https://dotmaptiles.createlab.org/tilesv1/%23f1f823;acs2016_5year_tract2010.B03001_003;;%23f5f5f5;acs2016_5year_tract2010.B02001_002;;%23d94801;acs2016_5year_tract2010.B02001_003;;%2300fb21;acs2016_5year_tract2010.B02001_005%2Bacs2016_5year_tract2010.B02001_006;;%2367a9cf;acs2016_5year_tract2010.B02001_004;;%23F062FF;acs2016_5year_tract2010.B02001_007%2Bacs2016_5year_tract2010.B02001_008'
layers['total_race_1990'] = 'https://dotmaptiles.createlab.org/tilesv1/%23ffb300;census1990_block2010.P0080001;;%236eaaf1;census1990_block2010.P0100001;;%235eff00;census1990_block2010.P0100002;;%23ff0400;census1990_block2010.P0100004;;%2388533e;census1990_block2010.P0100003%2Bcensus1990_block2010.P0100005'
layers['total_race_2000'] = 'https://dotmaptiles.createlab.org/tilesv1/%23ffb300;census2000_block2010.P0080010;;%236eaaf1;census2000_block2010.P0080003;;%235eff00;census2000_block2010.P0080004;;%23ff0400;census2000_block2010.P0080006%2Bcensus2000_block2010.P0080007;;%2388533e;census2000_block2010.P0080005%2Bcensus2000_block2010.P0080008%2Bcensus2000_block2010.P0080009'
layers['total_race_2010'] = 'https://dotmaptiles.createlab.org/tilesv1/%23ffb300;census2010_block2010.P0090002;;%236eaaf1;census2010_block2010.P0090005;;%235eff00;census2010_block2010.P0090006;;%23ff0400;census2010_block2010.P0090008%2Bcensus2010_block2010.P0090009;;%2388533e;census2010_block2010.P0090007%2Bcensus2010_block2010.P0090010%2Bcensus2010_block2010.P0090009'
layers['total_race_2016'] = 'https://dotmaptiles.createlab.org/tilesv1/%23ffb300;acs2016_5year_tract2010.B03001_003;;%236eaaf1;acs2016_5year_tract2010.B02001_002;;%235eff00;acs2016_5year_tract2010.B02001_003;;%23ff0400;acs2016_5year_tract2010.B02001_005%2Bacs2016_5year_tract2010.B02001_006;;%2388533e;acs2016_5year_tract2010.B02001_004%2Bacs2016_5year_tract2010.B02001_007%2Bacs2016_5year_tract2010.B02001_008'

layers['acs2016_public_assistance'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ff00;acs2016_5year_tract2010.B09010_004;;%23ff0000;(acs2016_5year_tract2010.B09010_005%2Bacs2016_5year_tract2010.B09010_006)'
layers['homeownloss_2000_2010'] = 'https://dotmaptiles.createlab.org/tilesv1/%230066ff;min(census2000_block2010.H0110002%2Ccensus2010_block2010.H0110002%2Bcensus2010_block2010.H0110003);;%23ff0088;census2000_block2010.H0110002-(census2010_block2010.H0110002%2Bcensus2010_block2010.H0110003)'
layers['own_rent_1990'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census1990_block2010.H0200001;;%2300FFFF;census1990_block2010.H0200002'
layers['own_rent_2000'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census2000_block2010.H0110002;;%2300FFFF;census2000_block2010.H0110003'
layers['own_rent_2010'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census2010_block2010.H0110002%2Bcensus2010_block2010.H0110003;;%2300FFFF;census2010_block2010.H0110004'
layers['own_rent_black_1990'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census1990_block2010.H0090002;;%2300FFFF;census1990_block2010.H0090007'
layers['own_rent_black_2010'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census2010_block2010.H0140004;;%2300FFFF;census2010_block2010.H0140012'
layers['own_rent_white_1990'] = 'https://dotmaptiles.createlab.org/tilesv1/%23D40084;census1990_block2010.H0090001;;%2300FFFF;census1990_block2010.H0090006'

layers['acs2016_asian_origin'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ffff;acs2016_5year_tract2010.B02015_002%2Bacs2016_5year_tract2010.B02015_003%2Bacs2016_5year_tract2010.B02015_004%2Bacs2016_5year_tract2010.B02015_005%2Bacs2016_5year_tract2010.B02015_016%2Bacs2016_5year_tract2010.B02015_018%2Bacs2016_5year_tract2010.B02015_019;;%2300ff00;acs2016_5year_tract2010.B02015_006%2Bacs2016_5year_tract2010.B02015_013%2Bacs2016_5year_tract2010.B02015_021%2Bacs2016_5year_tract2010.B02015_022%2Bacs2016_5year_tract2010.B02015_009;;%23ffff00;acs2016_5year_tract2010.B02015_008%2Bacs2016_5year_tract2010.B02015_010%2Bacs2016_5year_tract2010.B02015_014;;%23ff0000;acs2016_5year_tract2010.B02015_007%2Bacs2016_5year_tract2010.B02015_011%2Bacs2016_5year_tract2010.B02015_012%2Bacs2016_5year_tract2010.B02015_015%2Bacs2016_5year_tract2010.B02015_017%2Bacs2016_5year_tract2010.B02015_020;;%23ffffff;acs2016_5year_tract2010.B02015_023%2Bacs2016_5year_tract2010.B02015_024%2Bacs2016_5year_tract2010.B02015_025'
layers['acs2016_income'] = 'https://dotmaptiles.createlab.org/tilesv1/%230000ff;acs2016_5year_tract2010.B25118_012%2Bacs2016_5year_tract2010.B25118_013;;%2300ff00;acs2016_5year_tract2010.B25118_010%2Bacs2016_5year_tract2010.B25118_011;;%23ffff00;acs2016_5year_tract2010.B25118_007%2Bacs2016_5year_tract2010.B25118_008%2Bacs2016_5year_tract2010.B25118_009;;%23ff0000;acs2016_5year_tract2010.B25118_003%2Bacs2016_5year_tract2010.B25118_004%2Bacs2016_5year_tract2010.B25118_005%2Bacs2016_5year_tract2010.B25118_006'
layers['acs2016_owner_housing_costs'] = 'https://dotmaptiles.createlab.org/tilesv1/%230000ff;acs2016_5year_tract2010.B25106_004%2Bacs2016_5year_tract2010.B25106_008%2Bacs2016_5year_tract2010.B25106_012%2Bacs2016_5year_tract2010.B25106_016%2Bacs2016_5year_tract2010.B25106_020;;%2300ff00;acs2016_5year_tract2010.B25106_005%2Bacs2016_5year_tract2010.B25106_009%2Bacs2016_5year_tract2010.B25106_013%2Bacs2016_5year_tract2010.B25106_017%2Bacs2016_5year_tract2010.B25106_021;;%23ff0000;acs2016_5year_tract2010.B25106_006%2Bacs2016_5year_tract2010.B25106_010%2Bacs2016_5year_tract2010.B25106_014%2Bacs2016_5year_tract2010.B25106_018%2Bacs2016_5year_tract2010.B25106_022%2Bacs2016_5year_tract2010.B25106_023'
layers['acs2016_renter_housing_costs'] = 'https://dotmaptiles.createlab.org/tilesv1/%230000ff;acs2016_5year_tract2010.B25106_026%2Bacs2016_5year_tract2010.B25106_030%2Bacs2016_5year_tract2010.B25106_034%2Bacs2016_5year_tract2010.B25106_038%2Bacs2016_5year_tract2010.B25106_042%2Bacs2016_5year_tract2010.B25106_046;;%2300ff00;acs2016_5year_tract2010.B25106_027%2Bacs2016_5year_tract2010.B25106_031%2Bacs2016_5year_tract2010.B25106_035%2Bacs2016_5year_tract2010.B25106_039%2Bacs2016_5year_tract2010.B25106_043;;%23ff0000;acs2016_5year_tract2010.B25106_028%2Bacs2016_5year_tract2010.B25106_032%2Bacs2016_5year_tract2010.B25106_036%2Bacs2016_5year_tract2010.B25106_040%2Bacs2016_5year_tract2010.B25106_044%2Bacs2016_5year_tract2010.B25106_045'
layers['acs2016_high_housing_costs'] = 'https://dotmaptiles.createlab.org/tilesv1/%23ffff00;acs2016_5year_tract2010.B25106_006%2Bacs2016_5year_tract2010.B25106_010%2Bacs2016_5year_tract2010.B25106_014%2Bacs2016_5year_tract2010.B25106_018%2Bacs2016_5year_tract2010.B25106_022%2Bacs2016_5year_tract2010.B25106_023;;%23ff0000;acs2016_5year_tract2010.B25106_028%2Bacs2016_5year_tract2010.B25106_032%2Bacs2016_5year_tract2010.B25106_036%2Bacs2016_5year_tract2010.B25106_040%2Bacs2016_5year_tract2010.B25106_044%2Bacs2016_5year_tract2010.B25106_045'
layers['acs2016_work'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ff00;acs2016_5year_tract2010.B12006_004%2Bacs2016_5year_tract2010.B12006_005%2Bacs2016_5year_tract2010.B12006_015%2Bacs2016_5year_tract2010.B12006_016%2Bacs2016_5year_tract2010.B12006_026%2Bacs2016_5year_tract2010.B12006_027%2Bacs2016_5year_tract2010.B12006_037%2Bacs2016_5year_tract2010.B12006_038%2Bacs2016_5year_tract2010.B12006_048%2Bacs2016_5year_tract2010.B12006_049;;%23ff0000;acs2016_5year_tract2010.B12006_006%2Bacs2016_5year_tract2010.B12006_007%2Bacs2016_5year_tract2010.B12006_017%2Bacs2016_5year_tract2010.B12006_018%2Bacs2016_5year_tract2010.B12006_028%2Bacs2016_5year_tract2010.B12006_029%2Bacs2016_5year_tract2010.B12006_039%2Bacs2016_5year_tract2010.B12006_040%2Bacs2016_5year_tract2010.B12006_050%2Bacs2016_5year_tract2010.B12006_051'

layers['acs2016_public_assistance'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ff00;acs2016_5year_tract2010.B09010_004;;%23ff0000;(acs2016_5year_tract2010.B09010_005%2Bacs2016_5year_tract2010.B09010_006)'
layers['acs2016_renter_w_children'] = 'https://dotmaptiles.createlab.org/tilesv1/%2300ff00;acs2016_5year_tract2010.B25115_018;;%23ff0000;acs2016_5year_tract2010.B25115_022%2Bacs2016_5year_tract2010.B25115_025'
layers['acs2016_renter_movein'] = 'https://dotmaptiles.createlab.org/tilesv1/%230000ff;acs2016_5year_tract2010.B25026_015;;%2300ff00;acs2016_5year_tract2010.B25026_013%2Bacs2016_5year_tract2010.B25026_014;;%23ffff00;acs2016_5year_tract2010.B25026_012;;%23ff3300;acs2016_5year_tract2010.B25026_011;;%23ffffff;acs2016_5year_tract2010.B25026_010'

In [16]:
pool = SimpleProcessPoolExecutor(4)

for name in sorted(layers.keys()):
    src_pattern = layers[name] + '/{level}/{x}/{y}.bin'
    dest_dir = '/workspace/rsargent/dotmaptiles/' + name
    max_level = 10
    pool.submit(mirror_tiles, src_pattern, dest_dir, max_level)

pool.shutdown()
None

/workspace/rsargent/dotmaptiles/acs2016_asian_origin/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_income/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_high_housing_costs/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_public_assistance/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_owner_housing_costs/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_renter_housing_costs/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_renter_movein/complete already exists, skipping
/workspace/rsargent/dotmaptiles/acs2016_work/complete already exists, skipping
/workspace/rsargent/dotmaptiles/homeownloss_2000_2010/complete already exists, skipping
/workspace/rsargent/dotmaptiles/own_rent_1990/complete already exists, skipping
/workspace/rsargent/dotmaptiles/own_rent_2000/complete already exists, skipping
/workspace/rsargent/dotmaptiles/own_rent_20

In [ ]:
!du -sh /workspace/rsargent/dotmaptiles